In [1]:
import Heuristic_model
import pymongo
import config

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
uri = "mongodb://celebal:bZPUhXkQDcdWioAIiwECCdSEiZL3zmQ6bojzYjdiDxQlHhBgzKrJjiuYCWtEbSB4QcinajhByNwKbWlsRoBQ0A==@celebal.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@celebal@"
client = pymongo.MongoClient(uri)
mongo_data1 = client.get_database(name="geography_sizing").get_collection(name='company_data')
mongo_data1

Collection(Database(MongoClient(host=['celebal.mongo.cosmos.azure.com:10255'], document_class=dict, tz_aware=False, connect=True, replicaset='globaldb', retrywrites=False, maxidletimems=120000, appname='@celebal@', tls=True), 'geography_sizing'), 'company_data')

In [3]:
lc = config.Local_config()
obj1 = Heuristic_model.Model(lc.CENSUS_AGE_PATH,lc.CENSUS_INCOME_PATH,lc.CENSUS_POP_PATH)

In [4]:
import pickle
outputs = []
cursor = mongo_data1.find({})
c=0
assumed_revenue = 100
for document in cursor:
    c += 1
    res = {}
    res['_id'] = document['_id']
    if 'error' in document['Linkedin'].keys():
        res['error'] = 'company not found on linkedin'
    elif document['Linkedin']['sales_data'] == None:
        res['error'] = 'Sales data not available'
    #elif :

    else:
        Country_Dist,State_Dist = obj1.Distribute(document['Linkedin']['sales_data'], document['Linkedin']['locations_data'], document['Glassdoor'],document['Website'],assumed_revenue)
        #res['Country_Dist'] = Country_Dist
        #res['State_Dist'] = State_Dist
        #pickle.dump(res, open('outputs/Result.pkl', 'wb'))
        #print(document['_id'])
        outputs.append({"Country":Country_Dist,"State":State_Dist})
        if c > 60:
            break
pickle.dump(outputs , open("test_model.pkl","wb"))

In [5]:
df=pickle.load(open("test_model.pkl","rb"))

In [8]:
df[10]

{'Country': {'us': 100.0},
 'State': {'us': {'NY': 33.35628485054832,
   'CA': 33.55175091853799,
   'OH': 33.091964230913696}}}